In [2]:

import tensorflow as tf
%pip install mitdeeplearning 
import mitdeeplearning as mdl
import numpy as np
import time 
import os
import functools
from IPython import display as ipythondisplay
from tqdm import tqdm

# assert len(tf.config.list_physical_devices('GPU'))>0


Note: you may need to restart the kernel to use updated packages.


In [3]:
%apt-get install abcmidi timidity > /dev/null 2>&1

UsageError: Line magic function `%apt-get` not found.


In [4]:
# now download the data set
songs=mdl.lab1.load_training_data()

# print one of the song and inspect the details
example_song=songs[0]
print(example_song)

Found 817 songs in text
X:1
T:Alexander's
Z: id:dc-hornpipe-1
M:C|
L:1/8
K:D Major
(3ABc|dAFA DFAd|fdcd FAdf|gfge fefd|(3efe (3dcB A2 (3ABc|!
dAFA DFAd|fdcd FAdf|gfge fefd|(3efe dc d2:|!
AG|FAdA FAdA|GBdB GBdB|Acec Acec|dfaf gecA|!
FAdA FAdA|GBdB GBdB|Aceg fefd|(3efe dc d2:|!


In [8]:
# now convet into playable in system rather than abc notation
mdl.lab1.play_song(example_song)

In [10]:
# now our list of song into a single string containing all songs
songs_joined="\n\n".join(songs)

# find all unique character in the joined string
vocab=sorted(set(songs_joined))
print("there are",len(vocab),"unique characters in the dataset")

there are 83 unique characters in the dataset


now we need to crate a numerical representation of our text-based dataset
To do this, we will generate two lookup table that map charToNum and other is NumToChar


In [11]:
# define numerical representaion of text
# mapping from char to unique index
# to get index of char d 
# we need char2idx[d]
char2idx={
    u:i for i ,u in enumerate(vocab)
}

# create mapping form indices to char
# form unique indices to char in our vocb
idx2char=np.array(vocab)


In [16]:
# lets peek this numerical repersentation of our dataset
print("{")
for char,_ in zip(char2idx,range(20)):
    print('{:3s}: {:3d},'.format(repr(char),char2idx[char]))
print('....\n')    

{
'\n':   0,
' ':   1,
'!':   2,
'"':   3,
'#':   4,
"'":   5,
'(':   6,
')':   7,
',':   8,
'-':   9,
'.':  10,
'/':  11,
'0':  12,
'1':  13,
'2':  14,
'3':  15,
'4':  16,
'5':  17,
'6':  18,
'7':  19,
....



In [17]:
# now vectorize the songs string
def vectorize_string(string):
    vectorized_output=np.array([char2idx[char] for char in string])
    return vectorized_output
vectorized_songs=vectorize_string(songs_joined)

In [27]:
# now look at how the first part of the text is mapped to an integer represntaion
print('{}-----char mapped to int --->{}'.format(repr(songs_joined[:100]),vectorized_songs[:100]))
assert isinstance(vectorized_songs,np.ndarray),"return should be a numpy array"

"X:1\nT:Alexander's\nZ: id:dc-hornpipe-1\nM:C|\nL:1/8\nK:D Major\n(3ABc|dAFA DFAd|fdcd FAdf|gfge fefd|(3efe"-----char mapped to int --->[49 22 13  0 45 22 26 67 60 79 56 69 59 60 73  5 74  0 51 22  1 64 59 22
 59 58  9 63 70 73 69 71 64 71 60  9 13  0 38 22 28 82  0 37 22 13 11 20
  0 36 22 29  1 38 56 65 70 73  0  6 15 26 27 58 82 59 26 31 26  1 29 31
 26 59 82 61 59 58 59  1 31 26 59 61 82 62 61 62 60  1 61 60 61 59 82  6
 15 60 61 60]


In [32]:
# now we will break text into chunks of seq_len+1. suppose seq_lenght is 4 and out text is hello
# then out input seq is hell and target sequence ie ello
# batch definition  to creat trainning examlple
def get_batch(vectorized_songs,seq_length,batch_size):
    # length of the vectorize songs string
    n=vectorized_songs.shape[0] -1
    # randomly choose the statring indices for the examples in the training batch
    idx=np.random.choice(n-seq_length,batch_size)
    # construct a list of input squence for training batch
    input_batch=[vectorized_songs[i:i+seq_length] for i in idx]
    # construct a list of output sequecees for the training batch
    output_batch=[vectorized_songs[i+1:i+seq_length+1] for i in idx]
     
    # x_batch  and y_batch provied  the true input and target for the network of training
    x_batch=np.reshape(input_batch,[batch_size,seq_length])
    y_batch=np.reshape(output_batch,[batch_size,seq_length])
    return x_batch,y_batch
# perform some simple tests to make sure your batch fxn working properly
test_args=(vectorized_songs,10,2)
if  not mdl.lab1.test_batch_func_types(get_batch,test_args) or \
    not mdl.lab1.test_batch_func_shapes(get_batch,test_args) or \
    not mdl.lab1.test_batch_func_next_step(get_batch,test_args):
    print("=======\n[fail] could not pass the test")
else:
    print("========\n[pass] pass the test")



[PASS] test_batch_func_types
[PASS] test_batch_func_shapes
[PASS] test_batch_func_next_step
[pass] pass the test
